# **Taller Flask**


## 1. Despliegue de una API REST  

Para este apartado vamos a realizar un primer despliegue de un servicio web que simplemente contenga un mensaje. Para ello hay que seguir los siguientes pasos:  

1. Busca un lugar en tu ordenador y crea una carpeta de "proyectos_clase", donde alojarás todos los proyectos de estos talleres, y en su interior crea una carpeta llamada "taller_api_local".  

2. En su interior crea un nuevo script de python con nombre "app1.py'  

3. Introduce el código del taller siguiendo los pasos.  

Primero haremos la importación de la librería y crearemos la configuración de la API Flask.  

Después crearemos nuestro primer enrutado donde viene declarado el tipo de petición. En este caso será un GET a la URL del servicio raíz, el cual devuelve una "página web", un HTML básico con un par de frases.  

Finalmente, Flask levanta la instancia del servidor mediante "app.run()".

In [ ]:
from flask import Flask

app = Flask(__name__)
app.config["DEBUG"] = True

@app.route('/', methods=['GET'])
def home():
    return """
    <h1>Distant Reading Archive</h1>
    <p>This site is a prototype API for distant reading of science fiction novels.</p>
    """

app.run()

Para ejecutar el script, ve a un "GitBash", "Powershell", "Simbolo del sistema" (o terminal si estás
en MacOS) y dentro de la carpeta "/proyectos_clase/taller_api_local", ejecuta "python app1.py" (o "python3 app1.py" en MacOS/Linux).  

Esto lanzará el servicio en local. Accede al mismo mediante la URL sugerida por el output de la sentencia (lo normal es que sea http://127.0.0.1:5000/).  

Para que el servicio deje de correr pulsa CTRL + C.  

Este debería ser el output:  

![](img/output_1.png)

Fíjate en el terminal cada vez que accedes a esa URL. El servicio está recibiendo
peticiones, como si un usuario estuviera accediendo a una web.  

![](img/terminal_1.png)

### ¿Qué está haciendo Flask?  

Flask está mapeando la URL "/", con la función "home()", por lo que
cada vez que se acceda a la URL "/", es decir, sin ninguna ruta extra, llamará a la función
"home()" y devolverá su output, que en este caso es un texto HTML, pero podría ser un
JSON con otros datos.  

El proceso de mapear URLs a funciones se denomina **routing**.  

```python
@app.route('/', methods=['GET'])
```

En este caso nuestro routing utiliza como método GET, que es la acción HTTP para que el servidor devuelva datos
al usuario. Se suele combinar mucho con POST para recibir datos del usuario.  

```python
app.config["DEBUG"] = True
```

Se utiliza para que salten los errores en la página y podamos ver bien qué es. Si no, pondría
un "Bad Gateway".

## 2. API con datos
Como el propósito de este taller es montar una API con la que podamos acceder a una base
de datos de libros, comenzaremos creando algunos datos sintéticos, así como un enrutado
a los mismos:
1. Importa:

In [ ]:
from flask import request, jsonify

2. Declara una lista con varios diccionarios. Éstos serán los datos que devolverá la API

In [ ]:
books = [
    {'id': 0,
     'title': 'A Fire Upon the Deep',
     'author': 'Vernor Vinge',
     'first_sentence': 'The coldsleep itself was dreamless.',
     'year_published': '1992'},
    {'id': 1,
     'title': 'The Ones Who Walk Away From Omelas',
     'author': 'Ursula K. Le Guin',
     'first_sentence': 'With a clamor of bells that set the swallows soaring, the Festival of Summer came to the city Omelas, bright-towered by the sea.',
     'published': '1973'},
    {'id': 2,
     'title': 'Dhalgren',
     'author': 'Samuel R. Delany',
     'first_sentence': 'to wound the autumnal city.',
     'published': '1975'},
    {'id': 3,
     'title': 'The Chain',
     'author': 'Jaime G. Páramo',
     'first_sentence': 'There were tears on her eyes and fears trapped her mind but, inside, the courage of those who have nothing to lose and all to win, flown wild and free.',
     'published': '2025'}
]

3. Añade la ruta y función para acceder a estos datos

In [ ]:
@app.route('/api/v1/resources/books/all', methods=['GET'])
def api_all():
    return jsonify(books)

4. Corre la aplicación y accede a
http://127.0.0.1:5000/api/v1/resources/books/all  

Verás que la API devuelve un JSON, que es el formato de datos más común para
comunicaciones web. Este json lo creamos a partir de la lista de diccionarios, gracias
a la función "jsonify()", de flask.  

De momento nuestro programa tiene un punto de acceso y devuelve todos los libros de
nuestra "base de datos".

## 3. API para búsqueda de datos
Hasta el momento hemos creado un endpoint y una ruta de acceso a todos los datos. Para
este apartado implementaremos en la API una búsqueda de objetos por ID:
1. Añade la siguiente función para buscar por ID. Comprueba en los argumentos de la
petición si existe ID. De ser así, buscamos en la base de datos con ese ID, y en caso
contrario, devolvemos un mensaje de error.

In [ ]:
@app.route('/api/v1/resources/books', methods=['GET'])
def api_id():
    if 'id' in request.args:
        id = int(request.args['id'])
    else:
        return "Error: No id field provided. Please specify an id."
    results = []
    for book in books:
        if book['id'] == id:
            results.append(book)
    return jsonify(results)

Accede a las siguientes URLs para comprobar su funcionamiento:  

[127.0.0.1:5000/api/v1/resources/books?id=0](http://127.0.0.1:5000/api/v1/resources/books?id=0)  
[127.0.0.1:5000/api/v1/resources/books?id=1](http://127.0.0.1:5000/api/v1/resources/books?id=1)  
[127.0.0.1:5000/api/v1/resources/books?id=2](http://127.0.0.1:5000/api/v1/resources/books?id=2)  
[127.0.0.1:5000/api/v1/resources/books?id=3](http://127.0.0.1:5000/api/v1/resources/books?id=3)  

Todo lo que va después del "?", se denominan **query parameters**, y los empleamos para
filtrar un tipo de datos concreto.  

En este punto tenemos creado un nuevo enrutado:
"/api/v1/resources/books", que llamará a la función "api_id()", cada vez
que se acceda a esa ruta.  

Otra manera de acceder a los datos es con el argumento en la propia URL. Como en el
siguiente ejemplo donde vamos a buscar por título:

In [ ]:
@app.route('/api/v1/resources/books/<string:title>', methods=['GET'])
def get_by_title(title):
    for book in books:
        if book['title'] == title:
            return jsonify(book)
    return jsonify({'message': "Book not found"})

### **Inciso: Nuestra recomendación para pruebas de APIs -> Postman**

Para realizar peticiones a APIs que requieren enviar argumentos en el cuerpo de la petición (como en solicitudes POST, PUT, etc.), recomendamos utilizar **Postman**. Esta herramienta especializada facilita enormemente el desarrollo, prueba y documentación de APIs.

Postman permite:
- Realizar peticiones HTTP de todo tipo (GET, POST, PUT, DELETE, etc.)
- Configurar fácilmente headers, parámetros de consulta y cuerpos de petición
- Organizar peticiones en colecciones para proyectos
- Guardar y compartir configuraciones de API
- Automatizar pruebas con scripts

**Enlace de descarga**: [https://www.postman.com/downloads/](https://www.postman.com/downloads/)

Postman está disponible para Windows, macOS y Linux, y cuenta con una versión gratuita muy completa que cubre la mayoría de las necesidades para pruebas de API.

Ahora ya podemos continuar con nuestra API :)

### Búsqueda con el cuerpo de la petición

También podemos acceder a datos con los argumentos en el propio cuerpo de la petición HTTP. Fíjate que se ha cambiado la versión del enrutado, para no confundir con los anteriores.

In [ ]:
@app.route('/api/v2/resources/books', methods=['GET'])
def get_by_id():
    id = int(request.get_json()['id'])
    for book in books:
        if book['id'] == id:
            return jsonify(book)
    return jsonify({'message': "Book not found"})

### Añadiendo un libro con POST

Si quisiésemos subir a la base de datos algún libro, realizaremos un POST:

In [ ]:
@app.route('/api/v1/resources/books', methods=['POST'])
def post_book():
    data = request.get_json()
    books.append(data)
    return data

## 4. API con BD SQL
Finalmente crearemos una API que sea capaz de manejar errores de consulta a la base de
datos, descargarse todos los libros y filtrarlos por fecha de publicación.  

La BD se puede descargar desde [este enlace](https://programminghistorian.org/assets/creating-apis-with-python-and-flask/books.db) y utilizaremos el siguiente código:

In [ ]:
from flask import Flask, request
import sqlite3

app = Flask(__name__)
app.config["DEBUG"] = True


@app.route('/', methods=['GET'])
def home():
    return """
    <h1>Distant Reading Archive</h1>
    <p>This site is a prototype API for distant reading of science fiction novels.</p>
    """

@app.route('/api/v1/resources/books/all', methods=['GET'])
def get_all():
    connection = sqlite3.connect('data/books.db')
    cursor = connection.cursor()
    select_books = "SELECT * FROM books"
    result = cursor.execute(select_books).fetchall()
    connection.close()
    return {'books': result}

@app.route('/api/v1/resources/books/<string:author>', methods=['GET'])
def get_by_author(author):
    connection = sqlite3.connect('data/books.db')
    cursor = connection.cursor()
    select_books = "SELECT * FROM books WHERE author=?"
    result = cursor.execute(select_books, (author,)).fetchall()
    connection.close()
    return {'books': result}

@app.route('/api/v1/resources/books/filter', methods=['GET'])
def filter_table():
    query_parameters = request.get_json()
    id = query_parameters.get('id')
    published = query_parameters.get('published')
    author = query_parameters.get('author')
    connection = sqlite3.connect('data/books.db')
    cursor = connection.cursor()
    query = "SELECT * FROM books WHERE"
    to_filter = []
    if id:
        query += ' id=? AND'
        to_filter.append(id)
    if published:
        query += ' published=? AND'
        to_filter.append(published)
    if author:
        query += ' author=? AND'
        to_filter.append(author)
    if not (id or published or author):
        return "page not found 404"
    query = query[:-4] + ";"
    result = cursor.execute(query, to_filter).fetchall()
    connection.close()
    return {'books': result}

app.run()

### Probando la API con Base de Datos  

Para probar esta API, puedes acceder a las siguientes URLs:

http://127.0.0.1:5000/api/v1/resources/books/all  
http://127.0.0.1:5000/api/v1/resources/book/Connie%20Willis  

La base de datos tiene 67 entradas, con libros ganadores del premio Hugo entre 1953 y 2014. Incluye nombres de las novelas, autor, id, año de publicación y primera frase.

### Función dict_factory

Si quisieras devolver los resultados de SQL como diccionarios en lugar de tuplas, puedes utilizar la siguiente función:

In [ ]:
def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

# Y luego asignar esta función al "row_factory" de la conexión:
connection = sqlite3.connect('books.db')
connection.row_factory = dict_factory
cursor = connection.cursor()

Nuestra API con dict_factory() quedaría de la siguiente forma:

In [ ]:
from flask import Flask, request
import sqlite3

def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

app = Flask(__name__)
app.config["DEBUG"] = True

@app.route('/', methods=['GET'])
def home():
    return """
    <h1>Distant Reading Archive</h1>
    <p>This site is a prototype API for distant reading of science fiction novels.</p>
    """

@app.route('/api/v1/resources/books/all', methods=['GET'])
def get_all():
    connection = sqlite3.connect('data/books.db')
    connection.row_factory = dict_factory
    cursor = connection.cursor()
    select_books = "SELECT * FROM books"
    result = cursor.execute(select_books).fetchall()
    connection.close()
    return {'books': result}

@app.route('/api/v1/resources/books/<string:author>', methods=['GET'])
def get_by_author(author):
    connection = sqlite3.connect('data/books.db')
    connection.row_factory = dict_factory
    cursor = connection.cursor()
    select_books = "SELECT * FROM books WHERE author=?"
    result = cursor.execute(select_books, (author,)).fetchall()
    connection.close()
    return {'books': result}

@app.route('/api/v1/resources/books/filter', methods=['GET'])
def filter_table():
    query_parameters = request.get_json()
    id = query_parameters.get('id')
    published = query_parameters.get('published')
    author = query_parameters.get('author')
    connection = sqlite3.connect('data/books.db')
    connection.row_factory = dict_factory
    cursor = connection.cursor()
    query = "SELECT * FROM books WHERE"
    to_filter = []
    if id:
        query += ' id=? AND'
        to_filter.append(id)
    if published:
        query += ' published=? AND'
        to_filter.append(published)
    if author:
        query += ' author=? AND'
        to_filter.append(author)
    if not (id or published or author):
        return "page not found 404"
    query = query[:-4] + ";"
    result = cursor.execute(query, to_filter).fetchall()
    connection.close()
    return {'books': result}

app.run()

## 5. Material extra  

Si quieres seguir formándote en Flask y APIs REST, estos recursos te pueden ayudar:  

1. Tutorial completo y nivel avanzado  
    https://blog.miguelgrinberg.com/post/the-flask-mega-tutorial-part-i-hello-world  

2. REST APIs Videocurso - O'Reilly  
    https://learning.oreilly.com/videos/rest-apis-with/9781788621526/  

3. Flask Web Development - O'Reilly  
    https://learning.oreilly.com/library/view/flask-web-development/9781491991725/  
